In [1]:
# Import libraries

import sys
import logging
import pandas as pd
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent.parent
sys.path.append(str(root_path))

import paths

logging.basicConfig(level=logging.INFO)

In [2]:
land_path = paths.input_root / 'geo' / 'Markanvändning_Sveriges_Kommuner.xlsx'
land_use = pd.read_excel(land_path)

# Prune the data frame
land_use.drop(land_use.columns[0], axis=1, inplace=True) # Drop column 1
land_use.columns = land_use.iloc[1].values  # Set the second row as header
land_use = land_use.drop([0,1])  # Drop the row with new header values
land_use.rename(columns={land_use.columns[0]: 'Kommun+kod'}, inplace=True)  # Rename based on column index
land_use = land_use.dropna(subset=['Kommun+kod']) # Drop all the bottom text that's not data
land_use[['Kod', 'Namn']] = land_use['Kommun+kod'].str.split(' ', n=1, expand=True) # Split the kommunkod and kommun name
land_use = land_use.drop(columns=['Kommun+kod']) # Drop the old combined column
land_use.set_index('Kod', inplace=True) # Set the index to be kommunkod
land_use = land_use[['Namn'] + [col for col in land_use.columns if col != 'Namn']] # Reorder the columns so that Kommun comes first

land_use

,Namn,åkermark,betesmark,total jordbruksmark,"skogsmark, produktiv","skogsmark, improduktiv",total skogsmark,bebyggd och anlagd mark,öppen myrmark,övrig mark,total landareal,total vattenareal
Kod,,,,,,,,,,,,
0114,Upplands Väsby,1228,158,1386,2744,216,2960,1632,136,1424,7539,888
0115,Vallentuna,7284,872,8156,17764,2581,20345,3097,598,3768,35964,1086
0117,Österåker,1646,435,2081,16218,3376,19595,5187,470,4070,31402,24359
0120,Värmdö,1028,578,1606,18387,8228,26616,8400,605,7428,44654,255255
0123,Järfälla,198,186,384,1702,158,1860,1868,72,1224,5407,929
...,...,...,...,...,...,...,...,...,...,...,...,...
2580,Luleå,7252,289,7541,152038,20905,172943,8881,13390,7903,210658,284765
2581,Piteå,7956,85,8041,216495,40612,257107,8287,28223,9330,310988,156747
2582,Boden,5270,245,5515,308061,52373,360434,6625,28522,3514,404610,27960


In [3]:
building_path = paths.input_root / 'geo' / 'Markanvändning_byggnader_Sveriges_Kommuner.xlsx'
building_use = pd.read_excel(building_path)

# Prune the data frame
building_use.drop(building_use.columns[0], axis=1, inplace=True) # Drop column 1
building_use.columns = building_use.iloc[1].values  # Set the second row as header
building_use = building_use.drop([0,1])  # Drop the row with new header values
building_use.rename(columns={building_use.columns[0]: 'Kommun+kod'}, inplace=True)  # Rename based on column index
building_use = building_use.dropna(subset=['Kommun+kod']) # Drop all the bottom text that's not data
building_use[['Kod', 'Namn']] = building_use['Kommun+kod'].str.split(' ', n=1, expand=True) # Split the kommunkod and kommun name
building_use = building_use.drop(columns=['Kommun+kod']) # Drop the old combined column
building_use.set_index('Kod', inplace=True) # Set the index to be kommunkod
building_use = building_use[['Namn'] + [col for col in building_use.columns if col != 'Namn']] # Reorder the columns so that Kommun comes first
for column in building_use[['bostad', 'industri', 'samhällsfunktion', 'verksamhet', 'ekonomibyggnad', 'komplementbyggnad', 'övrig byggnad', 'okänt ändamål', 'totalt']].columns: # Convert strings to numbers
    building_use[column] = pd.to_numeric(building_use[column], errors='coerce')
building_use[['bostad', 'industri', 'samhällsfunktion', 'verksamhet', 'ekonomibyggnad', 'komplementbyggnad', 'övrig byggnad', 'okänt ändamål', 'totalt']] = building_use[['bostad', 'industri', 'samhällsfunktion', 'verksamhet', 'ekonomibyggnad', 'komplementbyggnad', 'övrig byggnad', 'okänt ändamål', 'totalt']] / 10 # Convert from 1000 m2 to hectares
building_use['ej_bostad'] = building_use[['industri', 'samhällsfunktion', 'verksamhet', 'ekonomibyggnad', 'komplementbyggnad', 'övrig byggnad', 'okänt ändamål', 'totalt']].sum(axis=1)
building_use.rename(columns={column: f"byggnad_{column}" for column in building_use.columns}, inplace=True)
building_use


,byggnad_Namn,byggnad_bostad,byggnad_industri,byggnad_samhällsfunktion,byggnad_verksamhet,byggnad_ekonomibyggnad,byggnad_komplementbyggnad,byggnad_övrig byggnad,byggnad_okänt ändamål,byggnad_totalt,byggnad_ej_bostad
Kod,,,,,,,,,,,
0114,Upplands Väsby,121.6,23.9,19.1,21.4,1.7,40.3,7.0,NaN,235.0,348.4
0115,Vallentuna,134.8,21.2,17.2,6.3,6.3,88.0,3.1,NaN,276.9,419.0
0117,Österåker,243.2,12.3,22.9,9.9,0.4,130.9,0.9,NaN,420.4,597.7
0120,Värmdö,323.3,14.0,20.3,20.7,2.4,185.0,25.0,NaN,590.5,857.9
0123,Järfälla,193.3,41.4,32.0,39.1,NaN,54.6,5.7,NaN,366.2,539.0
...,...,...,...,...,...,...,...,...,...,...,...
2580,Luleå,355.3,104.1,66.1,49.8,1.7,268.4,4.8,NaN,850.3,1345.2
2581,Piteå,240.7,70.4,38.0,25.3,0.9,245.1,15.7,NaN,636.0,1031.4
2582,Boden,182.0,18.2,41.8,20.7,1.0,132.3,7.4,NaN,403.4,624.8


In [4]:
# Join the two dataframes

land_use = land_use.join(building_use.drop(['byggnad_Namn'], axis=1), how='left')

In [5]:
# Build the regions (only VGR for now)

kommunkoder_vgr = [
    '1460', '1438', '1439', '1484', '1461', '1430', '1421', '1427', '1486', '1435', '1488', '1485', '1487', '1492',
    '1440', '1489', '1480', '1401', '1482', '1441', '1462', '1481', '1402', '1415', '1419', '1407',
    '1443', '1490', '1466', '1463', '1465', '1452', '1491', '1442',
    '1445', '1499', '1444', '1447', '1471', '1497', '1446', '1494', '1493', '1495', '1496', '1472', '1498', '1473', '1470'
]

land_use.loc['14'] = land_use[land_use.index.isin(kommunkoder_vgr)].sum()
land_use.loc['14', 'Namn'] = 'Västra Götalands Län'

land_use

,Namn,åkermark,betesmark,total jordbruksmark,"skogsmark, produktiv","skogsmark, improduktiv",total skogsmark,bebyggd och anlagd mark,öppen myrmark,övrig mark,...,byggnad_bostad,byggnad_industri,byggnad_samhällsfunktion,byggnad_verksamhet,byggnad_ekonomibyggnad,byggnad_komplementbyggnad,byggnad_övrig byggnad,byggnad_okänt ändamål,byggnad_totalt,byggnad_ej_bostad
Kod,,,,,,,,,,,,,,,,,,,,,
0114,Upplands Väsby,1228,158,1386,2744,216,2960,1632,136,1424,...,121.6,23.9,19.1,21.4,1.7,40.3,7.0,NaN,235.0,348.4
0115,Vallentuna,7284,872,8156,17764,2581,20345,3097,598,3768,...,134.8,21.2,17.2,6.3,6.3,88.0,3.1,NaN,276.9,419.0
0117,Österåker,1646,435,2081,16218,3376,19595,5187,470,4070,...,243.2,12.3,22.9,9.9,0.4,130.9,0.9,NaN,420.4,597.7
0120,Värmdö,1028,578,1606,18387,8228,26616,8400,605,7428,...,323.3,14.0,20.3,20.7,2.4,185.0,25.0,NaN,590.5,857.9
0123,Järfälla,198,186,384,1702,158,1860,1868,72,1224,...,193.3,41.4,32.0,39.1,NaN,54.6,5.7,NaN,366.2,539.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2581,Piteå,7956,85,8041,216495,40612,257107,8287,28223,9330,...,240.7,70.4,38.0,25.3,0.9,245.1,15.7,NaN,636.0,1031.4
2582,Boden,5270,245,5515,308061,52373,360434,6625,28522,3514,...,182.0,18.2,41.8,20.7,1.0,132.3,7.4,NaN,403.4,624.8
2583,Haparanda,1818,132,1950,58349,10531,68880,2952,13105,6161,...,65.1,10.9,10.3,11.6,2.3,61.1,5.5,NaN,166.7,268.4


In [6]:
# Build kommunförbunden (only the 4 in VGR for now)

fyrbodal = ['1460', '1438', '1439', '1484', '1461', '1430', '1421', '1427', '1486', '1435', '1488', '1485', '1487', '1492']
goteborgsregionen = ['1440', '1489', '1480', '1401', '1482', '1441', '1462', '1481', '1402', '1415', '1419', '1407']
sjuharad = ['1443', '1490', '1466', '1463', '1465', '1452', '1491', '1442']
skaraborg = ['1445', '1499', '1444', '1447', '1471', '1497', '1446', '1494', '1493', '1495', '1496', '1472', '1498', '1473', '1470']

land_use.loc['-'.join(fyrbodal)] = land_use[land_use.index.isin(fyrbodal)].sum()
land_use.loc['-'.join(fyrbodal), 'Namn'] = 'Fyrbodal'
land_use.loc['-'.join(goteborgsregionen)] = land_use[land_use.index.isin(goteborgsregionen)].sum()
land_use.loc['-'.join(goteborgsregionen), 'Namn'] = 'Göteborgsregionen'
land_use.loc['-'.join(sjuharad)] = land_use[land_use.index.isin(sjuharad)].sum()
land_use.loc['-'.join(sjuharad), 'Namn'] = 'Sjuhärad'
land_use.loc['-'.join(skaraborg)] = land_use[land_use.index.isin(skaraborg)].sum()
land_use.loc['-'.join(skaraborg), 'Namn'] = 'Skaraborg'

land_use


,Namn,åkermark,betesmark,total jordbruksmark,"skogsmark, produktiv","skogsmark, improduktiv",total skogsmark,bebyggd och anlagd mark,öppen myrmark,övrig mark,...,byggnad_bostad,byggnad_industri,byggnad_samhällsfunktion,byggnad_verksamhet,byggnad_ekonomibyggnad,byggnad_komplementbyggnad,byggnad_övrig byggnad,byggnad_okänt ändamål,byggnad_totalt,byggnad_ej_bostad
Kod,,,,,,,,,,,,,,,,,,,,,
0114,Upplands Väsby,1228,158,1386,2744,216,2960,1632,136,1424,...,121.6,23.9,19.1,21.4,1.7,40.3,7.0,NaN,235.0,348.4
0115,Vallentuna,7284,872,8156,17764,2581,20345,3097,598,3768,...,134.8,21.2,17.2,6.3,6.3,88.0,3.1,NaN,276.9,419.0
0117,Österåker,1646,435,2081,16218,3376,19595,5187,470,4070,...,243.2,12.3,22.9,9.9,0.4,130.9,0.9,NaN,420.4,597.7
0120,Värmdö,1028,578,1606,18387,8228,26616,8400,605,7428,...,323.3,14.0,20.3,20.7,2.4,185.0,25.0,NaN,590.5,857.9
0123,Järfälla,198,186,384,1702,158,1860,1868,72,1224,...,193.3,41.4,32.0,39.1,NaN,54.6,5.7,NaN,366.2,539.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,Västra Götalands Län,458512,65141,523653,1284999,127002,1411999,165679,53014,237168,...,6615.6,2262.1,1086.3,934.0,189.3,5458.9,149.4,0.0,16695.1,26775.1
1460-1438-1439-1484-1461-1430-1421-1427-1486-1435-1488-1485-1487-1492,Fyrbodal,113054,10432,123486,446833,49049,495879,43581,15955,88381,...,1547.1,346.7,201.1,217.1,30.4,1379.1,40.6,0.0,3762.3,5977.3
1440-1489-1480-1401-1482-1441-1462-1481-1402-1415-1419-1407,Göteborgsregionen,32016,7756,39772,156058,12591,168647,44325,4861,52719,...,2745.3,919.6,477.5,439.7,20.4,1197.6,45.8,0.0,5845.5,8946.1


In [7]:
# Write the file

land_use.to_csv(paths.input_root / 'geo' / 'markanvandning.csv.gz', compression='gzip')